In [1]:
import openai

from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchValue

from langsmith import Client

In [4]:
import qdrant_client


qdrant_client = QdrantClient(url="http://localhost:6333")

In [5]:
all_points = qdrant_client.scroll(
    collection_name="amazon_items-collection-00",
    limit=100,
    offset=None,
    with_payload=True,
    with_vectors=False
)

In [6]:
all_points

([Record(id=0, payload={'description': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil [Na

In [7]:

all_points[0][0].payload

{'description': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil [Native Palm Rejection]- R

In [10]:
all_context = [{"id": data.payload["parent_asin"], "text": data.payload["description"]} for data in all_points[0]]

In [11]:
all_context


[{'id': 'B0BF18F6R7',
  'text': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil [Native Pa

### Render a prompt to generate synthetic Eval Reference dataset

In [20]:
import json

output_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "Suggested question."
            },
            "chunk_ids": {
                "type": "array",
                "description": "List of chunk ids that are relevant to answer the question."
            },
            "answer_example": {
                "type": "string",
                "description": "Suggested answer to the question grounded in the context."
            },
            "reasoning": {
                "type": "string",
                "description": "Short explanation of why the question can be answered using the provided chunks."
            }
        }
    }
}

SYSTEM_PROMPT = f"""
I am building a RAG application. I have a collection on 50 chunks of text.
The RAG application will act as a shopping assistant that can answer questions about the stock of products we have available.
I will provide all of the available products to you with IDs of each chunk.
I want you to come up with 30 questions to which answers could be grounded in the chunk context.
As an output I need you to provide me the list of questions and the IDs of the chunks that could be used to answer them.
Also, provide an example answer to the question given the context of the chunks.
Also, provide the reason why you chose the chunks to answer the questions.
Construct 15 questions that could use multiple chunks to answer.
Construct 10 questions that could be answered using a single chunk.
Construct 5 questions that can't be answered using the provided chunks.

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema, indent=2)}
</OUTPUT JSON SCHEMA>

I need to be able to parse the json output.

"""

USER_PROMPT = f"""
Here is the context of the chunks, each list element is a dictionary with the id of the chunk and the text:
{all_context}
"""





In [21]:
print(SYSTEM_PROMPT)
print(USER_PROMPT)


I am building a RAG application. I have a collection on 50 chunks of text.
The RAG application will act as a shopping assistant that can answer questions about the stock of products we have available.
I will provide all of the available products to you with IDs of each chunk.
I want you to come up with 30 questions to which answers could be grounded in the chunk context.
As an output I need you to provide me the list of questions and the IDs of the chunks that could be used to answer them.
Also, provide an example answer to the question given the context of the chunks.
Also, provide the reason why you chose the chunks to answer the questions.
Construct 15 questions that could use multiple chunks to answer.
Construct 10 questions that could be answered using a single chunk.
Construct 5 questions that can't be answered using the provided chunks.

<OUTPUT JSON SCHEMA>
{
  "type": "array",
  "items": {
    "type": "object",
    "properties": {
      "question": {
        "type": "string",

In [22]:
response = openai.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ]
)

print(response.choices[0].message.content)

```json
[
  {
    "question": "Which earbud products are compatible with both iPhone and Android devices and offer waterproof protection?",
    "chunk_ids": [
      "B0C6K1GQCF",
      "B0CBMPG524",
      "B0B9FTVL58"
    ],
    "answer_example": "The TELSOR Wireless Earbuds (B0C6K1GQCF) are compatible with most Bluetooth-enabled iPhones and Android devices, offer IPX7 waterproof protection, and have a long playtime. Similarly, the Open Ear Headphones (B0CBMPG524) provide IPX7 waterproofing, comfort, and are suitable for use with various devices. The Wireless Earbuds (B0B9FTVL58) also offer Bluetooth 5.3 connectivity, are compatible with smartphones, and have IPX7 waterproofing.",
    "reasoning": "These chunks describe wireless earbuds/headphones that explicitly mention both compatibility with iPhone/Android and IPX7 waterproof ratings."
  },
  {
    "question": "Which charging cables are Apple MFi Certified and compatible with iPhone 13?",
    "chunk_ids": [
      "B0BYYLJRHT",
     

In [23]:
import json


json_output = response.choices[0].message.content
json_output = json_output.replace("```json", "").replace("```", "")
json_output = json.loads(json_output)
json_output

[{'question': 'Which earbud products are compatible with both iPhone and Android devices and offer waterproof protection?',
  'chunk_ids': ['B0C6K1GQCF', 'B0CBMPG524', 'B0B9FTVL58'],
  'answer_example': 'The TELSOR Wireless Earbuds (B0C6K1GQCF) are compatible with most Bluetooth-enabled iPhones and Android devices, offer IPX7 waterproof protection, and have a long playtime. Similarly, the Open Ear Headphones (B0CBMPG524) provide IPX7 waterproofing, comfort, and are suitable for use with various devices. The Wireless Earbuds (B0B9FTVL58) also offer Bluetooth 5.3 connectivity, are compatible with smartphones, and have IPX7 waterproofing.',
  'reasoning': 'These chunks describe wireless earbuds/headphones that explicitly mention both compatibility with iPhone/Android and IPX7 waterproof ratings.'},
 {'question': 'Which charging cables are Apple MFi Certified and compatible with iPhone 13?',
  'chunk_ids': ['B0BYYLJRHT', 'B09TNXY54Y', 'B0BV6PWVCG'],
  'answer_example': 'The iPhone Charger 

In [28]:
len(json_output)

36

In [24]:
points = qdrant_client.scroll(
    collection_name="amazon_items-collection-00",
    scroll_filter=Filter(
        must=[
            FieldCondition(key="parent_asin", match=MatchValue(value="B09WCFC5D9"))
        ]
    ),
    limit=100,
    offset=None,
    with_payload=True,
    with_vectors=False
)

In [25]:
points

([Record(id=21, payload={'description': "RUSAM GA33 A Enhanced Edition TWS Game Earbuds True Wireless Stereo Headset Low Latency Bluetooth 5.2 Headphones Wireless Deep Bass Touch Control Earbuds HD Physical Noise Cancellation Earphones,White 【Advanced Bluetooth 5.2 Technology】RUSAM GA33 A built-in Bluetooth 5.2 chip,stable connection within 10 meters, Even if you cover your ears with your hands, you won't be disconnected,smooth like a wired headset. 【Game/Music Mode】The game/music mode can be switched by key operation to meet the needs of different usage scenarios.In music mode,13mm Dual Coil Dynamic Drivers，strong bass effect,heavy and low frequency all around the ear, shock more penetrating. 【Low Latency Game Mode】45ms low latency,there is a sound effect adjustment function in the game mode,and the subtle sound can also be captured to clearly distinguish the enemy's position. 【Long battery life】Built-in 40MAH battery in the headset, 400MAH battery in the charging box, can listen to m

In [27]:
def get_description(parent_asin: str) -> str:
    points = qdrant_client.scroll(
        collection_name="amazon_items-collection-00",
        scroll_filter=Filter(
            must=[
                FieldCondition(key="parent_asin", match=MatchValue(value=parent_asin))
            ]
        ),
        limit=100,
        offset=None,
        with_payload=True,
        with_vectors=False
    )   
    return points[0][0].payload["description"]

get_description("B09WCFC5D9")


"RUSAM GA33 A Enhanced Edition TWS Game Earbuds True Wireless Stereo Headset Low Latency Bluetooth 5.2 Headphones Wireless Deep Bass Touch Control Earbuds HD Physical Noise Cancellation Earphones,White 【Advanced Bluetooth 5.2 Technology】RUSAM GA33 A built-in Bluetooth 5.2 chip,stable connection within 10 meters, Even if you cover your ears with your hands, you won't be disconnected,smooth like a wired headset. 【Game/Music Mode】The game/music mode can be switched by key operation to meet the needs of different usage scenarios.In music mode,13mm Dual Coil Dynamic Drivers，strong bass effect,heavy and low frequency all around the ear, shock more penetrating. 【Low Latency Game Mode】45ms low latency,there is a sound effect adjustment function in the game mode,and the subtle sound can also be captured to clearly distinguish the enemy's position. 【Long battery life】Built-in 40MAH battery in the headset, 400MAH battery in the charging box, can listen to music continuously for 3-4 hours, and the

### Create Eval dataset in langsmith

In [30]:
import os
client = Client(api_key=os.getenv("LANGCHAIN_API_KEY"))

In [31]:
dataset_name = "rag-eval-dataset"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic evaluation dataset for RAG application"
)





#### TODO: My evalutated json did not have any halucinated IDs - but we need to prompt engineer this to ensure - no IDs are created like that, also add checks to ensure code does not break if invalid ID is there. 

In [32]:
for item in json_output:
    client.create_example(
        dataset_id=dataset.id,
        inputs={"question": item["question"]},
        outputs={
            "ground_truth": item["answer_example"],
            "reference_context_ids": item["chunk_ids"],
            "reference_context_text":[get_description(chunk_id) for chunk_id in item["chunk_ids"]]
        }
    )
